In [1]:
year = 2018
month = 12
start_date = None
end_date = None

In [2]:
import aiohttp

In [3]:
import asyncio
import json
import pandas as pd
import re
import papermill as pm
import datetime

In [4]:
target_date = datetime.date(year=year, month=month, day=1)

In [5]:
target_date = target_date.strftime("%Y%m")

In [6]:
target_date

'201812'

In [7]:
async def fetch(session, url):
    async with session.get(url) as response:
        return await response.text()

In [8]:
async with aiohttp.ClientSession() as session:
    json_str = await fetch(session, f"http://api.cntv.cn/lanmu/videolistByColumnId?id=TOPC1451557646802924&n=90&of=fdate&p=1&type=0&serviceId=tvcctv&d={target_date}")

In [9]:
json_data = json.loads(json_str)['response']['docs']

In [10]:
df = pd.DataFrame.from_records(json_data)

In [11]:
df.columns

Index(['videoBrief', 'videoChannel', 'videoDeleteFlg', 'videoEditMode',
       'videoFirstclass', 'videoFocusDate', 'videoId', 'videoKeyFrameUrl',
       'videoKeyFrameUrl2', 'videoKeyFrameUrl3', 'videoLastmodifyDate',
       'videoLength', 'videoMediaType', 'videoPlaystyle',
       'videoProductiontime', 'videoPublishFlg', 'videoSecondclass',
       'videoSharedCode', 'videoTag', 'videoThirdclass', 'videoTitle',
       'videoUrl'],
      dtype='object')

### 从标题中抽取，移除所有非数字 ###

In [12]:
df['videoRecordDay'] = df['videoTitle'].map(lambda x: re.sub("[^0-9]", "", x)).apply(pd.to_datetime)

In [13]:
df.set_index('videoRecordDay', inplace=True)

In [14]:
df.sort_index(inplace=True)

In [15]:
df.loc[start_date:end_date, 'videoUrl']

videoRecordDay
2018-12-01    http://tv.cctv.com/2018/12/01/VIDEqBR9QDpARsgU...
2018-12-02    http://tv.cctv.com/2018/12/02/VIDEKfFvjA6CpUZk...
2018-12-03    http://tv.cctv.com/2018/12/03/VIDEgiw3auwyi03J...
2018-12-04    http://tv.cctv.com/2018/12/04/VIDEhyIbPtdyzJSP...
2018-12-05    http://tv.cctv.com/2018/12/05/VIDEMk8YGVSeT8jI...
2018-12-06    http://tv.cctv.com/2018/12/06/VIDELRtDhzLZ0mzq...
2018-12-07    http://tv.cctv.com/2018/12/07/VIDE583cAB994IwG...
2018-12-08    http://tv.cctv.com/2018/12/08/VIDEPsvu3dbpcSxH...
2018-12-09    http://tv.cctv.com/2018/12/09/VIDErLTbXUGhSKnk...
2018-12-10    http://tv.cctv.com/2018/12/10/VIDEsY2GTjtODpQf...
2018-12-11    http://tv.cctv.com/2018/12/11/VIDEeocB0tk8FvOl...
2018-12-12    http://tv.cctv.com/2018/12/12/VIDEWKBV2ShHOO8S...
2018-12-13    http://tv.cctv.com/2018/12/13/VIDEarmi0SCoCDCr...
2018-12-14    http://tv.cctv.com/2018/12/14/VIDEe8Tv6QFIAqxQ...
2018-12-15    http://tv.cctv.com/2018/12/15/VIDEeWE2vNJr7ewP...
2018-12-16    http://tv.c

In [16]:
pm.record("dn_list", df.loc[start_date:end_date, 'videoUrl'].tolist())